In [3]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

In [4]:
# Load the saved DataFrame from the file
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
# Load the saved DataFrame from the file
with open('y.pickle', 'rb') as f:
    y = pickle.load(f)

In [5]:
data

,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Source Port_0,Source Port_5,Source Port_20,...,NAT Destination Port_27017,NAT Destination Port_28215,NAT Destination Port_42873,NAT Destination Port_45711,NAT Destination Port_49001,NAT Destination Port_51883,NAT Destination Port_52862,NAT Destination Port_61002,NAT Destination Port_63128,NAT Destination Port_63752
0,177,94,83,2,30,1,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,4768,1600,3168,19,17,10,9,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,238,118,120,2,1199,1,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3327,1438,1889,15,17,8,7,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,25358,6778,18580,31,16,13,18,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65527,314,192,122,6,15,4,2,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65528,4680740,67312,4613428,4675,77,985,3690,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65529,70,70,0,1,0,1,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
65530,70,70,0,1,0,1,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
y

(array([0, 0, 0, ..., 1, 1, 1]),
 Index(['allow', 'drop', 'deny', 'reset-both'], dtype='object'))

In [7]:
print(data.dtypes)

Bytes                         uint32
Bytes Sent                    uint32
Bytes Received                uint32
Packets                       uint32
Elapsed Time (sec)            uint32
                               ...  
NAT Destination Port_51883      bool
NAT Destination Port_52862      bool
NAT Destination Port_61002      bool
NAT Destination Port_63128      bool
NAT Destination Port_63752      bool
Length: 54958, dtype: object


### SVC

In [8]:
%%time
# select the boolean columns for training
bool_cols = data.iloc[:, 7:]
# select the non-boolean columns for training
non_bool_cols = ['Bytes', 'Bytes Sent', 'Bytes Received', 'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']

CPU times: user 1.74 s, sys: 2.5 s, total: 4.25 s
Wall time: 4.68 s


In [9]:
%%time
X_train_non_bool, X_test_non_bool, _, _ = train_test_split(data[non_bool_cols], y[0], test_size=0.4, stratify=y[0], random_state=42)

CPU times: user 38.9 ms, sys: 50.9 ms, total: 89.8 ms
Wall time: 137 ms


In [ ]:
%%time
# split the data into training and testing sets
X_train_bool, X_test_bool, y_train, y_test = train_test_split(data[bool_cols], y[0], test_size=0.4, stratify=y[0], random_state=42)

In [ ]:
%%time
# create a StandardScaler object and fit on the non-boolean training set only
scaler = StandardScaler()
scaler.fit(X_train_non_bool)

In [ ]:
%%time
# transform the non-boolean training and testing sets using the scaler
X_train_non_bool_scaled = scaler.transform(X_train_non_bool)
X_test_non_bool_scaled = scaler.transform(X_test_non_bool)

In [ ]:
%%time
# combine the scaled non-boolean training set with the boolean training set
X_train = pd.concat([pd.DataFrame(X_train_non_bool_scaled, columns=non_bool_cols), X_train_bool.reset_index(drop=True)], axis=1)
X_test = pd.concat([pd.DataFrame(X_test_non_bool_scaled, columns=non_bool_cols), X_test_bool.reset_index(drop=True)], axis=1)

In [ ]:
%%time

svm = SVC(C=1.0, kernel='rbf', degree=2, gamma='scale', cache_size=500)

# train the classifier using the sample data and show a progress bar
for i in tqdm(range(100)):
    svm.fit(X_train, y_train)